# K Means POC experiment notebook 

## Experiment description
the notebook train simple m kmeans model. Model implemented  complying the ML framework



##### Jupyter helpers:

In [9]:
%reload_ext autoreload
%autoreload

Define imports

In [10]:

from kmeans_poc.data import KMeansSampleDataLoader
from kmeans_poc.models import KMeansModel
from kmeans_poc.data_processing import DataProcessor
from kmeans_poc.experimentation import MlflowExperimentation
from kmeans_poc.evaluation import Evaluator, EvaluationMetrics


## Load data
*replace MyDataLoader with your DataLoader implementation*

In [11]:
data_loader = KMeansSampleDataLoader(dataset_name="generated",dataset_version=1.0 )
x = data_loader.get_dataset()

Define experimentation object, which will be used for logging the experiments parameters, metrics and artifacts
*Replace MlflowExperimentation if you use a different experimentation system*

In [12]:
experimentation = MlflowExperimentation(tracking_uri="databricks")

Create model/logic:
Note: used default empty preprocessor, as there is no preprocessing for this model

In [15]:
my_model = KMeansModel()

Run experiment

In [16]:
from kmeans_poc import ExperimentRunner

experiment_runner = ExperimentRunner(
    model=my_model,
    X_train=x,
    y_train=None,
    X_test=x,
    y_test=None,
    data_loader=data_loader,
    log_experiment=True,
    experiment_logger=experimentation,
    evaluator=evaluator,
    experiment_name="/Experiment",
)

results = experiment_runner.run()
print(results)


INFO: '/Experiment' does not exist. Creating a new experiment
Logging package in /Users/katyamust/anaconda3/envs/cont_review/lib/python3.7/site-packages/kmeans_poc-0.2.0-py3.7.egg/kmeans_poc
n_clusters: 2
